# Data Cleaing Austin MetroBike Trips Dataset
This dataset was provided by data.austintexas.gov. This is a API over CapMetros bike data. The link is below

https://data.austintexas.gov/Transportation-and-Mobility/Austin-MetroBike-Trips/tyfh-5r8s/data 

## 1. Load the data

In [1]:
#Import pandas
import pandas as pd

In [2]:
#Read csv file
df = pd.read_csv("Austin_CapMetroBike_Trips (2).csv")
#Display head
df.head()

C:\Users\KRAyu\AppData\Local\Temp\ipykernel_27688\1188569334.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Austin_CapMetroBike_Trips (2).csv")


,Trip ID,Membership or Pass Type,Bicycle ID,Bike Type,Checkout Datetime,Checkout Date,Checkout Time,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Month,Year
0,16459203,Local365,37,classic,12/01/2017 09:51:41 AM,12/01/2017,9:51:41,2496.0,8th & Congress,2495.0,4th & Congress,3,12,2017
1,4106422,Local365,971,classic,03/13/2015 04:20:24 PM,03/13/2015,16:20:24,2495.0,4th/Congress,2495.0,4th/Congress,14,3,2015
2,3454605,Local365,29,classic,10/14/2014 08:33:54 AM,10/14/2014,8:33:54,2712.0,Toomey Rd @ South Lamar,2575.0,Riverside/South Lamar,22,10,2014
3,3454592,Local365,646,classic,10/14/2014 08:32:03 AM,10/14/2014,8:32:03,2552.0,3rd/West,2496.0,8th/Congress,29,10,2014
4,4106421,24 Hour Walk Up Pass,124,classic,03/13/2015 04:20:15 PM,03/13/2015,16:20:15,2498.0,Dean Keeton/Speedway,2549.0,South 1st/Riverside @ Long Center,10,3,2015


There seems to be mixed data types in column 9 (Return Kiosok ID). I'll explore this later for now let's get some general info about our dataset.

## 2. Checking Missing Data

In [3]:
#Check missing data
df.isnull().sum()

Trip ID                        0
Membership or Pass Type     5237
Bicycle ID                     0
Bike Type                      0
Checkout Datetime              0
Checkout Date                  0
Checkout Time                  0
Checkout Kiosk ID           4447
Checkout Kiosk                 0
Return Kiosk ID            31788
Return Kiosk                   0
Trip Duration Minutes          0
Month                          0
Year                           0
dtype: int64

### Return/Checkout Kiosk IDs  

The missing **Return Kiosk ID** and **Checkout Kiosk ID** might be the only values I can manually impute. To do this, I need to determine if the **kiosk locations** are consistently associated with specific **kiosk IDs** in the existing data.  

#### **Exploration Plan**  
Before imputing missing values, I will:  
1. **Check if each Kiosk Name corresponds to a unique Kiosk ID** in the available data.  
2. **Identify inconsistencies**, if any, where the same Kiosk Name has multiple IDs.  
3. **Use the most frequent or consistent ID** for each Kiosk Name to fill in missing values.  

This approach ensures that imputation is reliable and aligns with existing patterns in the data. Let's explore this further.  


In [4]:
#First create subset of Dataframe that only includs the checkout/return kiosk ID and locations
subset = df[['Checkout Kiosk', 'Checkout Kiosk ID', 'Return Kiosk', 'Return Kiosk ID']]

#Next temporarly remove duplicates and null values from the subset
clean_subset = subset.dropna().drop_duplicates()

#Sort the data so that the same locations are grouped together (Sort in ascending order)
clean_subset = clean_subset.sort_values(by=['Checkout Kiosk', 'Return Kiosk'])

#Check data
clean_subset

,Checkout Kiosk,Checkout Kiosk ID,Return Kiosk,Return Kiosk ID
1496006,10th & Red River,4051.0,10th & Red River,4051.0
1507569,10th & Red River,4051.0,10th & Red River,4051
1496887,10th & Red River,4051.0,11th & Salina,4055.0
1503251,10th & Red River,4051.0,13th & San Antonio,3635.0
1876353,10th & Red River,4051.0,17th & Guadalupe,2540
...,...,...,...,...
262917,Zilker Park West,1006.0,Waller & 6th St.,2536.0
182985,Zilker Park West,1006.0,West & 6th St.,2537.0
39141,Zilker Park West,1006.0,Zilker Park,2574.0
165015,Zilker Park West,1006.0,Zilker Park West,1006.0


This seems to show that the same locations, have the same Kiosk ID's. However, we can further check this by checking specific locations. 

In [5]:
clean_subset['Checkout Kiosk'].value_counts().head(3)

Checkout Kiosk
Plaza Saltillo         371
Zilker Park            370
Barton Springs Pool    344
Name: count, dtype: int64

In [6]:
clean_subset['Return Kiosk'].value_counts().head(3)

Return Kiosk
Plaza Saltillo         384
Zilker Park            373
Barton Springs Pool    347
Name: count, dtype: int64

Seems like Plaza Saltillo, Zilker Park, and Barton Springs will be good enough. 

In [7]:
#First check Plaza Saltillo
mask = clean_subset[clean_subset['Checkout Kiosk'] == 'Plaza Saltillo']
mask[['Checkout Kiosk', 'Checkout Kiosk ID']]

,Checkout Kiosk,Checkout Kiosk ID
1538701,Plaza Saltillo,2542.0
1877794,Plaza Saltillo,2542.0
216523,Plaza Saltillo,2542.0
461792,Plaza Saltillo,2542.0
1496018,Plaza Saltillo,2542.0
...,...,...
1514276,Plaza Saltillo,2542.0
19353,Plaza Saltillo,2542.0
216874,Plaza Saltillo,2542.0
1511399,Plaza Saltillo,2542.0


In [8]:
#First check Plaza Saltillo check for return as well
mask = clean_subset[clean_subset['Return Kiosk'] == 'Plaza Saltillo']
mask[['Return Kiosk', 'Return Kiosk ID']]

,Return Kiosk,Return Kiosk ID
1504578,Plaza Saltillo,2542.0
1510598,Plaza Saltillo,2542
216515,Plaza Saltillo,4051.0
487149,Plaza Saltillo,2542
1543012,Plaza Saltillo,2542
...,...,...
1514027,Plaza Saltillo,2542
12549,Plaza Saltillo,2542.0
217491,Plaza Saltillo,2574.0
1535573,Plaza Saltillo,2542


In [9]:
#Next check Zilker Park for Checkout
mask = clean_subset[clean_subset['Checkout Kiosk'] == 'Zilker Park']
mask[['Checkout Kiosk', 'Checkout Kiosk ID']]

,Checkout Kiosk,Checkout Kiosk ID
231040,Zilker Park,2574.0
1516641,Zilker Park,2574.0
611495,Zilker Park,2574.0
149125,Zilker Park,2574.0
1610662,Zilker Park,2574.0
...,...,...
164172,Zilker Park,2574.0
1531163,Zilker Park,2574.0
22,Zilker Park,2574.0
1507442,Zilker Park,2574.0


In [10]:
#Check for return
mask = clean_subset[clean_subset['Return Kiosk'] == 'Zilker Park']
mask[['Return Kiosk', 'Return Kiosk ID']]

,Return Kiosk,Return Kiosk ID
1494708,Zilker Park,2574.0
222630,Zilker Park,4051.0
639723,Zilker Park,2574.0
164334,Zilker Park,2574.0
1532389,Zilker Park,2574
...,...,...
119570,Zilker Park,2574.0
1592362,Zilker Park,2574
22,Zilker Park,2574.0
1507442,Zilker Park,2574


In [11]:
#Lastly check Barton Springs
mask = clean_subset[clean_subset['Checkout Kiosk'] == 'Barton Springs Pool']
mask[['Checkout Kiosk', 'Checkout Kiosk ID']]

,Checkout Kiosk,Checkout Kiosk ID
1486303,Barton Springs Pool,2572.0
1530749,Barton Springs Pool,2572.0
636593,Barton Springs Pool,2572.0
1545894,Barton Springs Pool,2572.0
1878585,Barton Springs Pool,2572.0
...,...,...
163977,Barton Springs Pool,2572.0
1511423,Barton Springs Pool,2572.0
1347,Barton Springs Pool,2572.0
49046,Barton Springs Pool,2572.0


In [12]:
#Check for return as well
mask = clean_subset[clean_subset['Return Kiosk'] == 'Barton Springs Pool']
mask[['Return Kiosk', 'Return Kiosk ID']]

,Return Kiosk,Return Kiosk ID
1506559,Barton Springs Pool,2572.0
1513846,Barton Springs Pool,2572
736110,Barton Springs Pool,2572
1553910,Barton Springs Pool,2572
860820,Barton Springs Pool,2572.0
...,...,...
1520184,Barton Springs Pool,2572
1794,Barton Springs Pool,2572.0
15187,Barton Springs Pool,2574.0
1511186,Barton Springs Pool,2572


**Before** I get any further it seems that most of the inconsistent entries in terms of kiosk location and kiosk ID are coming from the returns. Why might this be? Maybe its a data entry error or maybe something more. I think that there's a chance that return kiosk ID's might be missing because they were stolen or never returned. This would explain why `Return Kiosk ID` has much more missing values that `Checkout Kiosk ID`. 

In [13]:
mask = df[df['Return Kiosk'] == 'Stolen']
mask[['Return Kiosk ID', 'Return Kiosk']].head(10)

,Return Kiosk ID,Return Kiosk
3210,NaN,Stolen
5669,NaN,Stolen
15416,NaN,Stolen
56736,NaN,Stolen
163505,NaN,Stolen
163506,NaN,Stolen
253711,NaN,Stolen
264844,NaN,Stolen
273049,NaN,Stolen
309245,NaN,Stolen


In [14]:
mask = df[df['Checkout Kiosk'] == 'Stolen']
mask[['Checkout Kiosk ID', 'Checkout Kiosk']]

,Checkout Kiosk ID,Checkout Kiosk
442856,1001.0,Stolen


**Most null return Kiosk ID's were stolen**. In that case I'll need to replace the null value with something else. 

#### Handling Missing Kiosk IDs

After reviewing the top three kiosk locations, it appears that for the most part, the **Kiosk IDs** align with their respective **Checkout Kiosk** locations. However, a small number of mismatches were observed (about 1 out of every 10 rows checked). These mismatches might be due to **data entry errors**.

For **Return Kiosk IDs**, it seems that most of the missing data is due to bikes being **stolen**. This aligns with the idea that stolen bikes may not be returned to their original kiosks, leading to missing or inconsistent **Return Kiosk IDs**.

#### **Plan for Handling Missing or Mismatched Kiosk IDs**

1. **Imputation of Missing Checkout Kiosk IDs**:  
   - For missing `Checkout Kiosk ID` values, I will impute based on the most frequent ID for each **Checkout Kiosk** location. This assumes that each kiosk should have a consistent ID, and missing values are likely due to missing data rather than a true difference in IDs.
   - The imputation will be done using the **mode** (most frequent value) of the `Checkout Kiosk ID` for each **Checkout Kiosk** location.

2. **Handling Return Kiosk IDs**:  
   - For **missing `Return Kiosk ID` values** (likely due to stolen bikes), I will leave them as `0` (or another placeholder), since it's unlikely that the bikes were returned to a proper kiosk.
   - For the **mismatched `Return Kiosk ID` values**, I will use the same approach as the checkout kiosks and impute based on the most frequent ID for each **Return Kiosk** location, assuming the missing data is due to occasional data entry errors.

By following these steps, I can ensure that the kiosk IDs remain consistent and handle missing or mismatched values appropriately.

In [15]:
#First impute Checkout Kiosk ID.
group = df.groupby('Checkout Kiosk')['Checkout Kiosk ID']
group.first()

Checkout Kiosk
10th & Red River                     4051.0
10th/Red River                       4051.0
11th & Salina                        4055.0
11th & San Jacinto                   3291.0
11th/Congress @ The Texas Capitol    2497.0
                                      ...  
Waller & 6th St.                     2536.0
West & 6th St.                       2537.0
Zilker Park                          2574.0
Zilker Park West                     1006.0
cesar Chavez/Congress                3684.0
Name: Checkout Kiosk ID, Length: 201, dtype: float64

#### Handling Inconsistent Location Names

Upon reviewing the data, I noticed that some kiosk locations have inconsistencies in their naming conventions, such as "10th & Red River" and "10th/Red River". These variations might cause issues during the grouping and imputation process, as they would be treated as separate locations.

#### **Plan for Standardizing Location Names**

1. **Standardizing Kiosk Location Names**:  
   - To ensure consistency, I will standardize the names of the locations by replacing common variations (such as "&" with "/") across the **Checkout Kiosk** and **Return Kiosk** columns.
   - This can be done by using string replacement methods in Python, such as `str.replace()` or regex functions, to ensure that locations like "10th & Red River" and "10th/Red River" are treated as the same.

2. **Handling Inconsistent Location Data**:  
   - After standardizing the location names, I will proceed with the imputation of missing kiosk IDs and ensure that the data reflects the true mapping between locations and kiosk IDs.


In [16]:
#Strip any trailing spaces from location names
df['Checkout Kiosk'] = df['Checkout Kiosk'].str.strip()
df['Return Kiosk'] = df['Return Kiosk'].str.strip()

In [17]:
#Check out unique names for both checkout and return locations
return_loc = df['Return Kiosk'].unique()
return_loc.sort()
return_loc

array(['10th & Red River', '10th/Red River', '11th & Salina',
       '11th & San Jacinto', '11th/Congress @ The Texas Capitol',
       '11th/Salina', '11th/San Jacinto',
       '12th/San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th/San Antonio', '13th/Trinity',
       '13th/Trinity @ Waterloo Greenway', '16th/San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '17th/Guadalupe',
       '21st & Speedway @PCL', '21st & University', '21st/Guadalupe',
       '21st/Speedway @ PCL', '21st/University', '22.5/Rio Grande',
       '22nd & Pearl', '22nd/Pearl', '23rd & Rio Grande',
       '23rd & San Jacinto @ DKR Stadium', '23rd/Pearl',
       '23rd/Rio Grande', '23rd/San Gabriel',
       '23rd/San Jacinto @ DKR Stadium', '26th/Nueces', '28th/Rio',
       '28th/Rio Grande', '2nd & Congress', '2nd/Congress',
       '2nd/Lavaca @ City Hall', '3rd & West', '3rd/Nueces',
       '3rd/Trinity @ The Convention Center', '3rd/West',
       '4th & Congress',

In [18]:
#Check out unique names for both checkout and return locations
checkout_loc = df['Checkout Kiosk'].unique()
checkout_loc.sort()
checkout_loc

array(['10th & Red River', '10th/Red River', '11th & Salina',
       '11th & San Jacinto', '11th/Congress @ The Texas Capitol',
       '11th/Salina', '11th/San Jacinto',
       '12th/San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th/San Antonio', '13th/Trinity',
       '13th/Trinity @ Waterloo Greenway', '16th/San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '17th/Guadalupe',
       '21st & Speedway @PCL', '21st & University', '21st/Guadalupe',
       '21st/Speedway @ PCL', '21st/University', '22.5/Rio Grande',
       '22nd & Pearl', '22nd/Pearl', '23rd & Rio Grande',
       '23rd & San Jacinto @ DKR Stadium', '23rd/Pearl',
       '23rd/Rio Grande', '23rd/San Gabriel',
       '23rd/San Jacinto @ DKR Stadium', '26th/Nueces', '28th/Rio',
       '28th/Rio Grande', '2nd & Congress', '2nd/Congress',
       '2nd/Lavaca @ City Hall', '3rd & West', '3rd/Nueces',
       '3rd/Trinity @ The Convention Center', '3rd/West',
       '4th & Congress',

**Definently some inconsistenties in the locations for the Kiosk.** Let's begin fixing them

In [19]:
#Convert / to &
df['Checkout Kiosk'] = df['Checkout Kiosk'].str.replace('/', '&')
df['Return Kiosk'] = df['Return Kiosk'].str.replace('/', '&')

In [20]:
#Check results
return_loc_check  = df['Return Kiosk'].unique()
return_loc_check.sort()
return_loc_check

array(['10th & Red River', '10th&Red River', '11th & Salina',
       '11th & San Jacinto', '11th&Congress @ The Texas Capitol',
       '11th&Salina', '11th&San Jacinto',
       '12th&San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th&San Antonio', '13th&Trinity',
       '13th&Trinity @ Waterloo Greenway', '16th&San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '17th&Guadalupe',
       '21st & Speedway @PCL', '21st & University', '21st&Guadalupe',
       '21st&Speedway @ PCL', '21st&University', '22.5&Rio Grande',
       '22nd & Pearl', '22nd&Pearl', '23rd & Rio Grande',
       '23rd & San Jacinto @ DKR Stadium', '23rd&Pearl',
       '23rd&Rio Grande', '23rd&San Gabriel',
       '23rd&San Jacinto @ DKR Stadium', '26th&Nueces', '28th&Rio',
       '28th&Rio Grande', '2nd & Congress', '2nd&Congress',
       '2nd&Lavaca @ City Hall', '3rd & West', '3rd&Nueces',
       '3rd&Trinity @ The Convention Center', '3rd&West',
       '4th & Congress',

In [21]:
#Add space around '&' if missing
df['Checkout Kiosk'] = df['Checkout Kiosk'].str.replace(r'(\S)&(\S)', r'\1 & \2', regex=True)
df['Return Kiosk'] = df['Return Kiosk'].str.replace(r'(\S)&(\S)', r'\1 & \2', regex=True)

In [22]:
#Check results
return_loc_check  = df['Return Kiosk'].unique()
return_loc_check.sort()
return_loc_check

array(['10th & Red River', '11th & Congress @ The Texas Capitol',
       '11th & Salina', '11th & San Jacinto',
       '12th & San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th & Trinity',
       '13th & Trinity @ Waterloo Greenway', '16th & San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '21st & Guadalupe',
       '21st & Speedway @ PCL', '21st & Speedway @PCL',
       '21st & University', '22.5 & Rio Grande', '22nd & Pearl',
       '23rd & Pearl', '23rd & Rio Grande', '23rd & San Gabriel',
       '23rd & San Jacinto @ DKR Stadium', '26th & Nueces', '28th & Rio',
       '28th & Rio Grande', '2nd & Congress', '2nd & Lavaca @ City Hall',
       '3rd & Nueces', '3rd & Trinity @ The Convention Center',
       '3rd & West', '4th & Congress',
       '4th & Guadalupe @ Republic Square',
       '4th & Neches @ MetroRail Downtown', '4th & Sabine', '5th & Bowie',
       '5th & Campbell', '5th & Guadalupe @ Republic Square',
       '5th & San M

In [23]:
#Check results
checkout_loc_check  = df['Return Kiosk'].unique()
checkout_loc_check.sort()
checkout_loc_check

array(['10th & Red River', '11th & Congress @ The Texas Capitol',
       '11th & Salina', '11th & San Jacinto',
       '12th & San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th & Trinity',
       '13th & Trinity @ Waterloo Greenway', '16th & San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '21st & Guadalupe',
       '21st & Speedway @ PCL', '21st & Speedway @PCL',
       '21st & University', '22.5 & Rio Grande', '22nd & Pearl',
       '23rd & Pearl', '23rd & Rio Grande', '23rd & San Gabriel',
       '23rd & San Jacinto @ DKR Stadium', '26th & Nueces', '28th & Rio',
       '28th & Rio Grande', '2nd & Congress', '2nd & Lavaca @ City Hall',
       '3rd & Nueces', '3rd & Trinity @ The Convention Center',
       '3rd & West', '4th & Congress',
       '4th & Guadalupe @ Republic Square',
       '4th & Neches @ MetroRail Downtown', '4th & Sabine', '5th & Bowie',
       '5th & Campbell', '5th & Guadalupe @ Republic Square',
       '5th & San M

In [24]:
#Impute missing 'Checkout Kiosk ID' using the most frequent ID per location
df['Checkout Kiosk ID'] = df.groupby('Checkout Kiosk')['Checkout Kiosk ID'].transform(
    lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x
)

In [25]:
#Replace Return Kiosk ID with 0 where the Return Kiosk is 'Stolen'
df.loc[df['Return Kiosk'] == 'Stolen', 'Return Kiosk ID'] = 0

In [26]:
#Recheck missing values
df.isnull().sum()

Trip ID                        0
Membership or Pass Type     5237
Bicycle ID                     0
Bike Type                      0
Checkout Datetime              0
Checkout Date                  0
Checkout Time                  0
Checkout Kiosk ID            442
Checkout Kiosk                 0
Return Kiosk ID            31756
Return Kiosk                   0
Trip Duration Minutes          0
Month                          0
Year                           0
dtype: int64

There still seems to be some missing data for both Kiosk ID's.

In [27]:
#Check checkout kiosk id's with null values
missing_checkout_ids = df[df['Checkout Kiosk ID'].isna()]
missing_checkout_ids[['Checkout Kiosk ID', 'Checkout Kiosk']]

,Checkout Kiosk ID,Checkout Kiosk
1459,NaN,Repair Shop
1466,NaN,Ready for deployment
2169,NaN,Ready for deployment
3289,NaN,Ready for deployment
3607,NaN,Ready for deployment
...,...,...
1591298,NaN,Repair Shop
1653199,NaN,Repair Shop
1710760,NaN,Eeyore's 2018
1740930,NaN,Eeyore's 2017


In [28]:
#Let's find out which Checkout Kiosk has the most missing values
missing_checkout_ids['Checkout Kiosk'].value_counts()

Checkout Kiosk
Ready for deployment                           279
MapJam at Pan Am Park                           32
Springfest 2022                                 32
MapJam at French Legation                       26
MapJam at Hops & Grain Brewery                  17
Repair Shop                                     15
MapJam at Scoot Inn                             11
Mobile Station @ Boardwalk Opening Ceremony      9
Re-branding                                      5
Mobile Station @ Unplugged                       4
Customer Service                                 4
Marketing Event                                  4
Eeyore's 2018                                    2
Mobile Station                                   1
Eeyore's 2017                                    1
Name: count, dtype: int64

#### **Handling Missing for Checkouts**

Upon analyzing the data, it became clear that certain kiosks were associated with temporary events or stations, leading to missing `Checkout Kiosk ID` values. These kiosks are likely not relevant for the broader analysis, so to improve data quality, I decided to **drop rows** where these temporary kiosks had missing `Checkout Kiosk ID` values.

#### **Temporary Kiosks Identified:**
The following kiosks were identified as temporary or event-related:

- Ready for deployment
- MapJam at Pan Am Park
- Springfest 2022
- MapJam at French Legation
- MapJam at Hops & Grain Brewery
- Repair Shop
- MapJam at Scoot Inn
- Mobile Station @ Boardwalk Opening Ceremony
- Re-branding
- Mobile Station @ Unplugged
- Customer Service
- Marketing Event
- Eeyore's 2018
- Mobile Station
- Eeyore's 2017

#### **Plan for Handling Missing Data:**

1. **Dropping Rows for Temporary Kiosks**:
   - Rows where the `Checkout Kiosk` corresponds to one of the temporary kiosks and has a missing `Checkout Kiosk ID` value were dropped from the dataset.
   

In [29]:
#Define the kiosk names that are event-based or temporary
event_kiosks = [
    'Ready for deployment', 'MapJam at Pan Am Park', 'Springfest 2022',
    'MapJam at French Legation', 'MapJam at Hops & Grain Brewery',
    'Repair Shop', 'MapJam at Scoot Inn', 'Mobile Station @ Boardwalk Opening Ceremony',
    'Re-branding', 'Mobile Station @ Unplugged', 'Customer Service', 'Marketing Event',
    "Eeyore's 2018", 'Mobile Station', "Eeyore's 2017"
]

#Drop rows where 'Checkout Kiosk' is in the event_kiosks list and Checkout Kiosk ID is missing
df_cleaned = df[~((df['Checkout Kiosk'].isin(event_kiosks)) & df['Checkout Kiosk ID'].isna())]

#Check missing values
df_cleaned.isnull().sum()

Trip ID                        0
Membership or Pass Type     5236
Bicycle ID                     0
Bike Type                      0
Checkout Datetime              0
Checkout Date                  0
Checkout Time                  0
Checkout Kiosk ID              0
Checkout Kiosk                 0
Return Kiosk ID            31698
Return Kiosk                   0
Trip Duration Minutes          0
Month                          0
Year                           0
dtype: int64

Ok that deals with missing values in the Checkout Kiosk ID, let's deal with Return Kiosk ID next. 

In [30]:
#Filter data to display only rows where return kiosk id is null
missing_return_ids = df_cleaned[df_cleaned['Return Kiosk ID'].isna()]
missing_return_ids[['Return Kiosk ID', 'Return Kiosk']]

,Return Kiosk ID,Return Kiosk
364,NaN,Customer Service
1163,NaN,Customer Service
1368,NaN,Ready for deployment
1489,NaN,Ready for deployment
2245,NaN,Ready for deployment
...,...,...
1810821,NaN,Fantasy Zilker
1810828,NaN,Fantasy Zilker
1812010,NaN,Fantasy Zilker
1817203,NaN,Fantasy Zilker


In [31]:
#Check top locations with missing ID'set
missing_return_ids['Return Kiosk'].value_counts()

Return Kiosk
Dean Keeton & Speedway                                6551
21st & Speedway @ PCL                                 3702
26th & Nueces                                         1581
Dean Keeton & Whitis                                  1234
28th & Rio                                            1109
                                                      ... 
State Capitol Visitors Garage @ San Jacinto & 12th       1
Brazos & 6th                                             1
City Hall & Lavaca & 2nd                                 1
21st & Speedway @PCL                                     1
17th & Guadalupe                                         1
Name: count, Length: 119, dtype: int64

With the following code I'll check whether the Return Kiosk location have the same Return Kiosk ID. If so I'll group them together and impute using the mode, similar to what I did above with Checkout Kiosk ID. 

In [32]:
mask = df_cleaned[df_cleaned['Return Kiosk'] == 'Dean Keeton & Speedway']
mask[['Return Kiosk ID', 'Return Kiosk']]
mask['Return Kiosk ID'].value_counts()

Return Kiosk ID
2498.0    65904
3794.0    22252
3794       6174
3798.0      976
3838.0      246
7189.0      184
3797.0      128
3795.0      126
3799.0      122
7188.0      118
7125.0      106
2547.0      103
4938.0       65
2548.0       43
7341.0       30
3790.0       23
4050.0       22
4879.0       18
2707.0       11
2497.0        5
2570.0        5
4051.0        5
3619.0        5
2561.0        4
2539.0        4
7131.0        4
2499.0        4
3621.0        3
2575          3
2494.0        3
2571.0        3
2562.0        3
2552          2
2572.0        2
2495.0        2
2547          2
3685.0        2
2496.0        2
4052.0        2
3455.0        2
2537.0        1
3687.0        1
2569.0        1
2566.0        1
4699.0        1
3294.0        1
4062.0        1
2503.0        1
2563.0        1
4060.0        1
Name: count, dtype: int64

In [33]:
mask = df_cleaned[df_cleaned['Return Kiosk'] == '21st & Speedway @ PCL']
mask[['Return Kiosk ID', 'Return Kiosk']]
mask['Return Kiosk ID'].value_counts()

Return Kiosk ID
3798.0    98115
3798       4177
2498.0      813
3838.0      568
7189.0      479
          ...  
2569          1
3795          1
7189          1
4048          1
2539          1
Name: count, Length: 67, dtype: int64

In [34]:
mask = df_cleaned[df_cleaned['Return Kiosk'] == '26th & Nueces']
mask[['Return Kiosk ID', 'Return Kiosk']]
mask['Return Kiosk ID'].value_counts()

Return Kiosk ID
3838.0    37774
3838        949
3798.0      442
2498.0      201
2547.0      168
          ...  
2570          1
4060          1
4062          1
2504          1
4059          1
Name: count, Length: 68, dtype: int64

In [35]:
mask = df_cleaned[df_cleaned['Return Kiosk'] == 'Dean Keeton & Whitis']
mask[['Return Kiosk ID', 'Return Kiosk']]
mask['Return Kiosk ID'].value_counts()

Return Kiosk ID
3795.0    54396
3795       2887
3798.0      312
2498.0      131
7189.0      104
3799.0       58
3838.0       53
7125.0       46
7188.0       44
2547.0       40
3797.0       24
4879.0       23
4938.0       21
2548.0       20
7341.0       20
3790.0       12
2499.0        5
4054.0        4
2552.0        4
3685.0        4
3377.0        3
4050.0        3
3619.0        3
2503.0        3
2574.0        3
2575.0        3
2561.0        3
4699.0        3
3621.0        2
7253          2
3794.0        2
3684.0        2
2501.0        2
2497.0        2
2504.0        2
2495.0        2
2567.0        2
2496.0        2
2537.0        1
4051.0        1
2571.0        1
2542.0        1
3455.0        1
4055.0        1
2539.0        1
2566.0        1
4047.0        1
7189          1
4048.0        1
7188          1
2562          1
3294          1
7131          1
Name: count, dtype: int64

In [36]:
mask = df_cleaned[df_cleaned['Return Kiosk'] == '28th & Rio']
mask[['Return Kiosk ID', 'Return Kiosk']]
mask['Return Kiosk ID'].value_counts()

Return Kiosk ID
7189.0    8108
Name: count, dtype: int64

#### Imputing Missing Values for Return Kiosk ID

In this analysis, I encountered missing values in the `Return Kiosk ID` column. Upon inspection, I noticed that many of the missing `Return Kiosk ID` values were associated with specific `Return Kiosk` locations. The challenge was to find a reasonable approach for imputing these missing values, given that the `Return Kiosk ID` sometimes differed across the same location, indicating multiple kiosk IDs for the same location.

#### Step 1: Analyzing the Data

First, I examined the distribution of `Return Kiosk ID` values for the `Return Kiosk` locations with missing IDs. It became evident that in most locations, there was one dominant `Return Kiosk ID`, where one ID appeared significantly more frequently than others. For example, for locations like `Dean Keeton & Speedway`, there were multiple kiosk IDs with nearly equal occurrences, making it harder to define a clear "mode." However, for other locations, a single ID was overwhelmingly frequent.

#### Step 2: Imputation Approach

After considering the data distribution, I decided to use the **mode of the `Return Kiosk ID`** for each `Return Kiosk` location to impute the missing values. This approach works well because, in most cases, one kiosk ID is more frequent than others at each location, making it a reasonable assumption for imputation.

To address the variability at certain locations (like `Dean Keeton & Speedway`), I recognized that imputing with the mode for other locations still made sense because the mode was clear and consistent across those sites.

#### Step 3: Implementing the Imputation

To impute the missing values, I used the following steps:
1. **Grouped the data by `Return Kiosk`** and calculated the mode of `Return Kiosk ID` for each location.
2. **Imputed missing `Return Kiosk ID` values** by filling them with the mode for the respective location. This ensured that all missing values were filled consistently based on the most frequent `Return Kiosk ID` for each `Return Kiosk`.

Since the `Return Kiosk ID` is not a key variable in my analysis, and I don't need to analyze individual IDs, I am confident that this imputation method will provide an accurate and effective solution.

#### Conclusion

By imputing missing values using the mode of `Return Kiosk ID` for each `Return Kiosk` location, I ensured that the missing data was filled in a reasonable and consistent manner, while avoiding the complexity of dealing with cases like `Dean Keeton & Speedway` where multiple IDs exist.

In [37]:
#Impute missing 'Checkout Kiosk ID' using the most frequent ID per location
df_cleaned['Return Kiosk ID'] = df_cleaned.groupby('Return Kiosk')['Return Kiosk ID'].transform(
    lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x
)

C:\Users\KRAyu\AppData\Local\Temp\ipykernel_27688\775363792.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x
C:\Users\KRAyu\AppData\Local\Temp\ipykernel_27688\775363792.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Return Kiosk ID'] = df_cleaned.groupby('Return Kiosk')['Return Kiosk ID'].transform(


In [38]:
#Check missing values again
df_cleaned.isnull().sum()

Trip ID                       0
Membership or Pass Type    5236
Bicycle ID                    0
Bike Type                     0
Checkout Datetime             0
Checkout Date                 0
Checkout Time                 0
Checkout Kiosk ID             0
Checkout Kiosk                0
Return Kiosk ID             472
Return Kiosk                  0
Trip Duration Minutes         0
Month                         0
Year                          0
dtype: int64

In [39]:
missing_return_ids = df_cleaned[df_cleaned['Return Kiosk ID'].isna()]
missing_return_ids

,Trip ID,Membership or Pass Type,Bicycle ID,Bike Type,Checkout Datetime,Checkout Date,Checkout Time,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Month,Year
364,4103736,24 Hour Walk Up Pass,561,classic,03/13/2015 01:38:09 PM,03/13/2015,13:38:09,2494.0,2nd & Congress,NaN,Customer Service,39,3,2015
1163,4095624,24 Hour Walk Up Pass,869,classic,03/12/2015 04:50:07 PM,03/12/2015,16:50:07,2539.0,3rd & Trinity @ The Convention Center,NaN,Customer Service,29,3,2015
2434,4060058,Local365,440,classic,03/08/2015 08:37:30 PM,03/08/2015,20:37:30,3838.0,Bullock Museum @ Congress & MLK,NaN,Repair Shop,9,3,2015
3804,4029199,24 Hour Walk Up Pass,561,classic,03/04/2015 02:04:41 AM,03/04/2015,2:04:41,2565.0,6th & Trinity,NaN,Customer Service,10,3,2015
4978,3988208,24 Hour Walk Up Pass,354,classic,02/21/2015 08:06:04 PM,02/21/2015,20:06:04,2823.0,East 5th & Broadway @ Capital Metro HQ,NaN,MapJam at Scoot Inn,14,2,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810821,17573041,Local365,2204,classic,06/05/2018 08:31:20 AM,06/05/2018,8:31:20,2549.0,Long Center @ South 1st & Riverside,NaN,Fantasy Zilker,235,6,2018
1810828,17574538,Walk Up,341,classic,06/05/2018 11:19:09 AM,06/05/2018,11:19:09,2539.0,Convention Center & 3rd & Trinity,NaN,Fantasy Zilker,83,6,2018
1812010,17574629,Local365,922,classic,06/05/2018 11:28:05 AM,06/05/2018,11:28:05,3619.0,6th & Congress,NaN,Fantasy Zilker,6,6,2018
1817203,17547535,U.T. Student Membership,852,classic,06/02/2018 04:15:58 PM,06/02/2018,16:15:58,3792.0,22nd & Pearl,NaN,Fantasy Zilker,23,6,2018


In [40]:
#Find most frequent locations with missing values 
missing_return_ids['Return Kiosk'].value_counts()

Return Kiosk
Repair Shop                                    152
Customer Service                               147
Fantasy Zilker                                  33
MapJam at French Legation                       28
Springfest 2022                                 24
MapJam at Pan Am Park                           21
MapJam at Hops & Grain Brewery                  18
Mobile Station @ Unplugged                      13
Marketing Event                                 11
Mobile Station                                   8
MapJam at Scoot Inn                              7
Mobile Station @ Boardwalk Opening Ceremony      4
Eeyore's 2017                                    4
Earth Day ATX 2017                               2
Name: count, dtype: int64

I'll just drop the rest of the missing data for Return Kiosk ID since there's not that many. 

In [41]:
#List of locations with missing return kiosk IDs
drop_locations = [
    'Repair Shop', 'Customer Service', 'Fantasy Zilker', 
    'MapJam at French Legation', 'Springfest 2022', 
    'MapJam at Pan Am Park', 'MapJam at Hops & Grain Brewery', 
    'Mobile Station @ Unplugged', 'Marketing Event', 
    'Mobile Station', 'MapJam at Scoot Inn', 
    'Mobile Station @ Boardwalk Opening Ceremony', 'Eeyore\'s 2017', 
    'Earth Day ATX 2017'
]

#Drop rows for these specific locations with missing return kiosk ID
df_cleaned = df_cleaned[~df_cleaned['Return Kiosk'].isin(drop_locations)]

#Check result
df_cleaned.isnull().sum()


Trip ID                       0
Membership or Pass Type    5236
Bicycle ID                    0
Bike Type                     0
Checkout Datetime             0
Checkout Date                 0
Checkout Time                 0
Checkout Kiosk ID             0
Checkout Kiosk                0
Return Kiosk ID               0
Return Kiosk                  0
Trip Duration Minutes         0
Month                         0
Year                          0
dtype: int64

### Membership or Pass Type

In [42]:
df_cleaned['Membership or Pass Type'].unique()

array(['Local365', '24 Hour Walk Up Pass', 'Local30', 'Weekender',
       'Walk Up', '7-Day', 'Local31', 'Explorer', 'Annual ',
       'Semester Membership', 'Founding Member', 'RideScout Single Ride',
       'Annual Membership', '3-Day Weekender', 'Weekender ($15 plus tax)',
       'Local365 ($80 plus tax)', 'Local30 ($11 plus tax)', 'Annual',
       'Annual Pass', 'RideScout Single Tide', 'Annual Pass (Original)',
       'Heartland Pass (Annual Pay)', 'Student Membership',
       'Annual Member', 'FunFunFun Fest 3 Day Pass',
       'Local365+Guest Pass', 'ACL Weekend Pass Special',
       'Try Before You Buy Special', 'Pay-as-you-ride',
       'Local365 Youth with helmet (age 13-17 riders)', 'Annual Plus',
       'Republic Rider (Annual)', 'U.T. Student Membership',
       'Denver B-cycle Founder', nan, 'HT Ram Membership',
       'Single Trip Ride', 'Single Trip (Pay-as-you-ride)',
       '3-Day Explorer', 'Single Trip', '$1 Pay by Trip Fall Special',
       'Annual Membership ', 'L

In [43]:
df_cleaned['Membership or Pass Type'].value_counts()

Membership or Pass Type
Student Membership                                    430345
Local365                                              389282
U.T. Student Membership                               289096
24 Hour Walk Up Pass                                  272591
Walk Up                                               260013
Local31                                               162154
Explorer                                              120218
Pay-as-you-ride                                        82730
Local30                                                55772
3-Day Weekender                                        51766
Weekender                                              40499
Single Trip (Pay-as-you-ride)                          35318
Local365+Guest Pass                                    13999
Single Trip                                            11708
Founding Member                                         6078
3-Day Explorer                                          6016


#### Imputing Missing Values in the 'Membership/Pass Type' Column

For the missing values in the **Membership/Pass Type** column, rather than making assumptions about the type of membership based on the kiosk location (such as assuming students use bikes near the university), I chose to impute the missing values with a neutral placeholder value: **'Unknown'**. 

This approach avoids any potential bias or misinterpretation that could arise from assuming certain types of users based on their locations. By replacing missing values with **'Unknown'**, the analysis can continue without making unwarranted assumptions about the data.


In [44]:
#Impute missing 'Membership/Pass Type' with 'Unknown'
df_cleaned['Membership or Pass Type'] = df_cleaned['Membership or Pass Type'].fillna('Unknown')


In [45]:
#Check to see if there's any more missing data
df_cleaned.isnull().sum()

Trip ID                    0
Membership or Pass Type    0
Bicycle ID                 0
Bike Type                  0
Checkout Datetime          0
Checkout Date              0
Checkout Time              0
Checkout Kiosk ID          0
Checkout Kiosk             0
Return Kiosk ID            0
Return Kiosk               0
Trip Duration Minutes      0
Month                      0
Year                       0
dtype: int64

## 3. Standardizing & Cleaning Columns

After addressing the missing data, the next crucial step in preparing the dataset for analysis is to **standardize and clean the columns**. This ensures that all columns are formatted consistently, making the data easier to analyze and model. Below are the key steps taken for standardizing and cleaning the dataset:

**1. Standardize Column Names**
   - The column names were standardized to ensure consistency and clarity. This included:
     - Converting all column names to lowercase.
     - Replacing spaces with underscores to make the column names more accessible.
     - Ensuring descriptive names for better readability.

**2. Remove or Rename Unnecessary Columns**
   - Any irrelevant or redundant columns were identified and removed from the dataset to improve analysis efficiency.
   - Columns with no significant contribution to the analysis were dropped.

**3. Fix Data Types**
   - Data types of columns were checked and adjusted where necessary:
     - Numeric columns were converted to appropriate numerical data types.
     - Date columns were formatted into datetime type to ensure proper date-based analysis.

**4. Clean String Columns**
   - String columns were cleaned to remove unwanted characters, extra spaces, and inconsistencies:
     - Leading and trailing spaces were removed.
     - String values were standardized, such as proper capitalization or case conversion.
     - Non-relevant characters were eliminated to ensure consistency.

**5. Ensure Consistent Date/Time Format**
   - For any columns with date or time values, the formats were checked and standardized to ensure they are consistent.
   - This helps avoid any discrepancies when performing date-based analysis or time-series modeling.

**6. Create New Features**
   - Extract hour, day of the week, etc from date columns. 



#### Clean Column Names

In [46]:
#Remove white spaces from column names
df_cleaned.columns = df_cleaned.columns.str.strip()

#Convert column names to lowercase
df_cleaned.columns = df_cleaned.columns.str.lower()

#Replace spaces with underscores
df_cleaned.columns = df_cleaned.columns.str.replace(' ', '_')

#Check column names
df_cleaned.columns

Index(['trip_id', 'membership_or_pass_type', 'bicycle_id', 'bike_type',
       'checkout_datetime', 'checkout_date', 'checkout_time',
       'checkout_kiosk_id', 'checkout_kiosk', 'return_kiosk_id',
       'return_kiosk', 'trip_duration_minutes', 'month', 'year'],
      dtype='object')

In [47]:
#Rename 'membership_or_pass_type' column
df_cleaned.rename(columns={
    'membership_or_pass_type': 'membership_type'
}, inplace=True)

In [48]:
df_cleaned.head()

,trip_id,membership_type,bicycle_id,bike_type,checkout_datetime,checkout_date,checkout_time,checkout_kiosk_id,checkout_kiosk,return_kiosk_id,return_kiosk,trip_duration_minutes,month,year
0,16459203,Local365,37,classic,12/01/2017 09:51:41 AM,12/01/2017,9:51:41,2496.0,8th & Congress,2495.0,4th & Congress,3,12,2017
1,4106422,Local365,971,classic,03/13/2015 04:20:24 PM,03/13/2015,16:20:24,2495.0,4th & Congress,2495.0,4th & Congress,14,3,2015
2,3454605,Local365,29,classic,10/14/2014 08:33:54 AM,10/14/2014,8:33:54,2712.0,Toomey Rd @ South Lamar,2575.0,Riverside & South Lamar,22,10,2014
3,3454592,Local365,646,classic,10/14/2014 08:32:03 AM,10/14/2014,8:32:03,2552.0,3rd & West,2496.0,8th & Congress,29,10,2014
4,4106421,24 Hour Walk Up Pass,124,classic,03/13/2015 04:20:15 PM,03/13/2015,16:20:15,2498.0,Dean Keeton & Speedway,2549.0,South 1st & Riverside @ Long Center,10,3,2015


#### Check/Fix Datatypes

In [49]:
#Check datatypes
df_cleaned.dtypes

trip_id                    int64
membership_type           object
bicycle_id                object
bike_type                 object
checkout_datetime         object
checkout_date             object
checkout_time             object
checkout_kiosk_id        float64
checkout_kiosk            object
return_kiosk_id           object
return_kiosk              object
trip_duration_minutes      int64
month                      int64
year                       int64
dtype: object

In [50]:
#Check out date columns
df_cleaned[['checkout_datetime', 'checkout_date', 'checkout_time']].head()

,checkout_datetime,checkout_date,checkout_time
0,12/01/2017 09:51:41 AM,12/01/2017,9:51:41
1,03/13/2015 04:20:24 PM,03/13/2015,16:20:24
2,10/14/2014 08:33:54 AM,10/14/2014,8:33:54
3,10/14/2014 08:32:03 AM,10/14/2014,8:32:03
4,03/13/2015 04:20:15 PM,03/13/2015,16:20:15


In [51]:
#Create new column time_of_day
#Define function
def time_of_day(x):
    if 'AM' in x:
        return 'AM'
    elif 'PM' in x:
        return 'PM'
    else:
        return None
    
#Apply to checkout_datetime column
df_cleaned['time_of_day'] = df_cleaned['checkout_datetime'].apply(time_of_day)

In [52]:
#Extract day_of_week from checkout_datetime column (String version)
df_cleaned['day_of_week'] = pd.to_datetime(df_cleaned['checkout_datetime']).dt.day_name()

In [53]:
#Convert remaining date columns to datetime data types
df_cleaned['checkout_date'] = pd.to_datetime(df_cleaned['checkout_date'], format='%m/%d/%Y')
df_cleaned['checkout_time'] = pd.to_datetime(df_cleaned['checkout_time'], format='%H:%M:%S')

In [54]:
#Get the day (friday, saturday, etc) from the checkout_date column
df_cleaned['checkout_day'] = df_cleaned['checkout_date'].dt.day

In [55]:
#Check result
df_cleaned[['checkout_datetime','checkout_date', 'checkout_time', 'time_of_day', 'day_of_week','checkout_day']].head()

,checkout_datetime,checkout_date,checkout_time,time_of_day,day_of_week,checkout_day
0,12/01/2017 09:51:41 AM,2017-12-01,1900-01-01 09:51:41,AM,Friday,1
1,03/13/2015 04:20:24 PM,2015-03-13,1900-01-01 16:20:24,PM,Friday,13
2,10/14/2014 08:33:54 AM,2014-10-14,1900-01-01 08:33:54,AM,Tuesday,14
3,10/14/2014 08:32:03 AM,2014-10-14,1900-01-01 08:32:03,AM,Tuesday,14
4,03/13/2015 04:20:15 PM,2015-03-13,1900-01-01 16:20:15,PM,Friday,13


In [56]:
#Extract hour from checkout_time
df_cleaned['hour'] = df_cleaned['checkout_time'].dt.hour

In [57]:
#Convert checkout_time back into a object
df_cleaned['checkout_time'] = pd.to_datetime(df_cleaned['checkout_time'], format='%H:%M:%S').dt.time

In [58]:
#Check results
df_cleaned[['checkout_datetime','checkout_date','checkout_time','time_of_day','day_of_week','checkout_day','hour']].head()

,checkout_datetime,checkout_date,checkout_time,time_of_day,day_of_week,checkout_day,hour
0,12/01/2017 09:51:41 AM,2017-12-01,09:51:41,AM,Friday,1,9
1,03/13/2015 04:20:24 PM,2015-03-13,16:20:24,PM,Friday,13,16
2,10/14/2014 08:33:54 AM,2014-10-14,08:33:54,AM,Tuesday,14,8
3,10/14/2014 08:32:03 AM,2014-10-14,08:32:03,AM,Tuesday,14,8
4,03/13/2015 04:20:15 PM,2015-03-13,16:20:15,PM,Friday,13,16


I found that some  `Return Kiosk ID's` contain "Event", which is a string value. For Return Kiosk ID to be a integer I need to deal with this. The location this ID corresponds with is ACL, so I'll give it a unique identifier of 2.  

In [59]:
#Change 'Event' values in Return Kiosk ID to 2, as it's unique identifer. 
df_cleaned['return_kiosk_id'].replace('Event','2',inplace=True)

C:\Users\KRAyu\AppData\Local\Temp\ipykernel_27688\3663518329.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['return_kiosk_id'].replace('Event','2',inplace=True)


In [60]:
#Change ID columns to integers
df_cleaned['checkout_kiosk_id'] = df_cleaned['checkout_kiosk_id'].astype(int)
df_cleaned['return_kiosk_id'] = df_cleaned['return_kiosk_id'].astype(int)

In [61]:
#Check datatypes
df_cleaned.dtypes

trip_id                           int64
membership_type                  object
bicycle_id                       object
bike_type                        object
checkout_datetime                object
checkout_date            datetime64[ns]
checkout_time                    object
checkout_kiosk_id                 int64
checkout_kiosk                   object
return_kiosk_id                   int64
return_kiosk                     object
trip_duration_minutes             int64
month                             int64
year                              int64
time_of_day                      object
day_of_week                      object
checkout_day                      int32
hour                              int32
dtype: object

In [62]:
#Define function to classify different time periods of the day based on traffic
def classify_time_period(x):
    if 7 <= x < 9:
        return 'Morning Commute'
    elif 11 <= x < 14:
        return 'Lunch Break'
    elif 14 <= x < 16:
        return 'Afternoon Surge'
    elif 16 <= x < 18:
        return 'Evening Rush'
    elif 18 <= x < 21:
        return 'Evening Leisure'
    elif 21 <= x < 24:
        return 'Late Night'
    else:
        return 'Off-Peak'
#Apply function
df_cleaned['time_period'] = df_cleaned['hour'].apply(classify_time_period)

In [63]:
#Check first 5 rows
df_cleaned.head()

,trip_id,membership_type,bicycle_id,bike_type,checkout_datetime,checkout_date,checkout_time,checkout_kiosk_id,checkout_kiosk,return_kiosk_id,return_kiosk,trip_duration_minutes,month,year,time_of_day,day_of_week,checkout_day,hour,time_period
0,16459203,Local365,37,classic,12/01/2017 09:51:41 AM,2017-12-01,09:51:41,2496,8th & Congress,2495,4th & Congress,3,12,2017,AM,Friday,1,9,Off-Peak
1,4106422,Local365,971,classic,03/13/2015 04:20:24 PM,2015-03-13,16:20:24,2495,4th & Congress,2495,4th & Congress,14,3,2015,PM,Friday,13,16,Evening Rush
2,3454605,Local365,29,classic,10/14/2014 08:33:54 AM,2014-10-14,08:33:54,2712,Toomey Rd @ South Lamar,2575,Riverside & South Lamar,22,10,2014,AM,Tuesday,14,8,Morning Commute
3,3454592,Local365,646,classic,10/14/2014 08:32:03 AM,2014-10-14,08:32:03,2552,3rd & West,2496,8th & Congress,29,10,2014,AM,Tuesday,14,8,Morning Commute
4,4106421,24 Hour Walk Up Pass,124,classic,03/13/2015 04:20:15 PM,2015-03-13,16:20:15,2498,Dean Keeton & Speedway,2549,South 1st & Riverside @ Long Center,10,3,2015,PM,Friday,13,16,Evening Rush


### Clean String Columns

In [64]:
#Find the string columns
df_cleaned.dtypes

trip_id                           int64
membership_type                  object
bicycle_id                       object
bike_type                        object
checkout_datetime                object
checkout_date            datetime64[ns]
checkout_time                    object
checkout_kiosk_id                 int64
checkout_kiosk                   object
return_kiosk_id                   int64
return_kiosk                     object
trip_duration_minutes             int64
month                             int64
year                              int64
time_of_day                      object
day_of_week                      object
checkout_day                      int32
hour                              int32
time_period                      object
dtype: object

String columns:
- membership_type
- bicycle_id
- bike_type
- checkout_datetime
- checkout_time 
- checkout_kiosk
- return_kiosk
- time_of_day
- day_of_week
- time_period

In [ ]:
#Define function for removing white spaces from string columns
def remove_whitespaces(x):
    return x.strip() if isinstance(x, str) else x 

#Apply to all string columns
df_cleaned = df_cleaned.apply(lambda col: col.apply(remove_whitespaces) if col.dtypes == 'object' else col)

#### Check unique values of each string columns
**Goal:**

Ensure consistency with locations or other naming conventions. If values are practically duplicates, I'll pick one of the names in order to standardize the value. 

In [66]:
#Membership type column 
membership_type = df_cleaned['membership_type'].unique()
membership_type.sort()
membership_type

array(['$1 Pay by Trip Fall Special', '$1 Pay by Trip Winter Special',
       '24 Hour Walk Up Pass', '3-Day Explorer', '3-Day Weekender',
       '7-Day', 'ACL 2019 Pass', 'ACL Weekend Pass Special',
       'Aluminum Access', 'Annual', 'Annual Member', 'Annual Membership',
       'Annual Pass', 'Annual Pass (30 minute)', 'Annual Pass (Original)',
       'Annual Plus', 'Annual Plus Membership', 'Denver B-cycle Founder',
       'Explorer', 'Explorer ($8 plus tax)', 'Founding Member',
       'FunFunFun Fest 3 Day Pass', 'HT Ram Membership',
       'Heartland Pass (Annual Pay)', 'Heartland Pass (Monthly Pay)',
       'Local30', 'Local30 ($11 plus tax)', 'Local31', 'Local365',
       'Local365 ($80 plus tax)', 'Local365 Youth (age 13-17 riders)',
       'Local365 Youth (age 13-17 riders)- 1/2 off Special',
       'Local365 Youth with helmet (age 13-17 riders)',
       'Local365+Guest Pass',
       'Local365+Guest Pass- 1/2 off Anniversary Special',
       'Local365- 1/2 off Anniversary Spec

**Changes to be made:** 
- Convert `Annual` and `Annual Member` to `Annual Membership`
- Convert `Membership: pay once  one-year commitment` and `Membership: pay once, one-year commitment` to `One-Year Membership`
- Convert `Single Trip` and `Single Trip (Pay-as-you-ride)` to `Single Trip Ride`
- Convert `U.T. Student Membership` to `UT Student Membership` 
- Maybe convert `Student Membership` to `UT Student Membership`, because a lot of the locations where the students membership is used is around UT's campus. 
- Convert `RideScout Single Tide` to `RideScout Single Ride`
- Convert `Annual Plus` to `Annual Plus Membership`

In [67]:
#Create dictionary to standard annual membership passes 
annual_membership = {
    'Annual': 'Annual Membership',
    'Annual Member': 'Annual Membership'
}

#Replace membership_type values
df_cleaned['membership_type'] = df_cleaned['membership_type'].replace(annual_membership)

#Create dictionary to standardize 1 year commitment 
one_year_commitment = {
    'Membership: pay once  one-year commitment': 'One Year Membership',
    'Membership: pay once, one-year commitment': 'One Year Membership'
}
#Replace values
df_cleaned['membership_type'] = df_cleaned['membership_type'].replace(one_year_commitment)

#Change 'RideScout Single Tide' to 'RideScout Single Ride' 
df_cleaned['membership_type'] = df_cleaned['membership_type'].replace('RideScout Single Tide', 'RideScout Single Ride')

#Change 'Annual Plus' to 'Annual Plus Membership
df_cleaned['membership_type'] = df_cleaned['membership_type'].replace('Annual Plus', 'Annual Plus Membership')

#Create dictionary to standardize 'Single Trip Ride'
single_trip_ride = {
    'Single Trip': 'Single Trip Ride',
       'Single Trip (Pay-as-you-ride)': 'Single Trip Ride'
}

#Replace values
df_cleaned['membership_type'] = df_cleaned['membership_type'].replace(single_trip_ride)

#Change 'U.T. Student Membership' to 'UT Student Membership' 
df_cleaned['membership_type'] = df_cleaned['membership_type'].replace('U.T. Student Membership', 'UT Student Membership')

In [68]:
#Recheck Membership type column 
membership_type = df_cleaned['membership_type'].unique()
membership_type.sort()
membership_type

array(['$1 Pay by Trip Fall Special', '$1 Pay by Trip Winter Special',
       '24 Hour Walk Up Pass', '3-Day Explorer', '3-Day Weekender',
       '7-Day', 'ACL 2019 Pass', 'ACL Weekend Pass Special',
       'Aluminum Access', 'Annual Membership', 'Annual Pass',
       'Annual Pass (30 minute)', 'Annual Pass (Original)',
       'Annual Plus Membership', 'Denver B-cycle Founder', 'Explorer',
       'Explorer ($8 plus tax)', 'Founding Member',
       'FunFunFun Fest 3 Day Pass', 'HT Ram Membership',
       'Heartland Pass (Annual Pay)', 'Heartland Pass (Monthly Pay)',
       'Local30', 'Local30 ($11 plus tax)', 'Local31', 'Local365',
       'Local365 ($80 plus tax)', 'Local365 Youth (age 13-17 riders)',
       'Local365 Youth (age 13-17 riders)- 1/2 off Special',
       'Local365 Youth with helmet (age 13-17 riders)',
       'Local365+Guest Pass',
       'Local365+Guest Pass- 1/2 off Anniversary Special',
       'Local365- 1/2 off Anniversary Special', 'Madtown Monthly',
       'One Year 

In [69]:
#Checkout Kiosk
checkout_kiosk = df_cleaned['checkout_kiosk'].unique()
checkout_kiosk.sort()
checkout_kiosk

array(['10th & Red River', '11th & Congress @ The Texas Capitol',
       '11th & Salina', '11th & San Jacinto',
       '12th & San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th & Trinity',
       '13th & Trinity @ Waterloo Greenway', '16th & San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '21st & Guadalupe',
       '21st & Speedway @ PCL', '21st & Speedway @PCL',
       '21st & University', '22.5 & Rio Grande', '22nd & Pearl',
       '23rd & Pearl', '23rd & Rio Grande', '23rd & San Gabriel',
       '23rd & San Jacinto @ DKR Stadium', '26th & Nueces', '28th & Rio',
       '28th & Rio Grande', '2nd & Congress', '2nd & Lavaca @ City Hall',
       '3rd & Nueces', '3rd & Trinity @ The Convention Center',
       '3rd & West', '4th & Congress',
       '4th & Guadalupe @ Republic Square',
       '4th & Neches @ MetroRail Downtown', '4th & Sabine', '5th & Bowie',
       '5th & Campbell', '5th & Guadalupe @ Republic Square',
       '5th & San M

In [ ]:
#Import libraries
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from itertools import combinations
import charset_normalizer

**Changes:**

In [71]:
#Capitialize cesar Chavez & Congress
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('cesar Chavez & Congress', 'Cesar Chavez & Congress')

#Convert `21st & Speedway @PCL` to `21st & Speedway @ PCL`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('21st & Speedway @PCL', '21st & Speedway @ PCL')

#Convert `Congress & Cesar Chavez` to `Cesar Chavez & Congress`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Congress & Cesar Chavez', 'Cesar Chavez & Congress')

#Convert `Riverside @ S. Lamar` to `Riverside & South Lamar`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Riverside @ S. Lamar', 'Riverside & South Lamar')

#Convert `South Congress & Barton Springs @ The Austin American-Statesman` to `South Congress & Barton Springs at the Austin American-Statesman`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('South Congress & Barton Springs at the Austin American-Statesman'
                                                                    ,'South Congress & Barton Springs @ The Austin American-Statesman')

#Convert `East 6th at Robert Martinez` to `East 6th & Robert T. Martinez`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('East 6th at Robert Martinez','East 6th & Robert T. Martinez')

#Convert `Sterzing at Barton Springs` to `Sterzing & Barton Springs`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Sterzing at Barton Springs', 'Sterzing & Barton Springs')

#Convert `28th & Rio` to `28th & Rio Grande`
df_cleaned['checkout_kisok'] = df_cleaned['checkout_kiosk'].replace('28th & Rio', '28th & Rio Grande')

#Standardize East 11th st. 
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('East 11th St. @ Victory Grill','East 11th @ Victory Grill')
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('East 11th St. & San Marcos', 'East 11th & San Marcos')
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('East 11th St & San Marcos', 'East 11th & San Marcos')
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kisok'].replace('East 11th St. at Victory Grill', 'East 11th @ Victory Grill')

#Convert `Lakeshore & Austin Hostel` to `Lakeshore @ Austin Hostel`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Lakeshore & Austin Hostel', 'Lakeshore @ Austin Hostel')

#Convert `Nueces & 3rd` to `Nueces @ 3rd`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Nueces & 3rd', 'Nueces @ 3rd')

#Standardize `Rainey @ Cummings`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Rainey & Cummings', 'Rainey @ Cummings')
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Rainey St @ Cummings', 'Rainey @ Cummings')

#Convert `East 6th & Pedernales` to `East 6th & Pedernales St.`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('East 6th & Pedernales','East 6th & Pedernales St.')

#Convert `Lake Austin Blvd & Deep Eddy` to `Lake Austin Blvd @ Deep Eddy`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Lake Austin Blvd & Deep Eddy', 'Lake Austin Blvd @ Deep Eddy')

#Convert `Sterzing & Barton Springs` to `Sterzing @ Barton Springs`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Sterzing & Barton Springs', 'Sterzing @ Barton Springs')
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Sterzing at Barton Springs', 'Sterzing @ Barton Springs')

#Convert `Barton Springs & Kinney` to `Barton Springs @ Kinney Ave`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kisok'].replace('Barton Springs & Kinney', 'Barton Springs @ Kinney Ave')

#Convert `Trinity & 6th Street` to `Trinity & 6th St`
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace('Trinity & 6th Street', 'Trinity & 6th St')

In [72]:
#Recheck Checkout Kiosk
checkout_kiosk = df_cleaned['checkout_kiosk'].unique()
checkout_kiosk.sort()
checkout_kiosk

array(['10th & Red River', '11th & Congress @ The Texas Capitol',
       '11th & Salina', '11th & San Jacinto',
       '12th & San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th & Trinity',
       '13th & Trinity @ Waterloo Greenway', '16th & San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '21st & Guadalupe',
       '21st & Speedway @ PCL', '21st & University', '22.5 & Rio Grande',
       '22nd & Pearl', '23rd & Pearl', '23rd & Rio Grande',
       '23rd & San Gabriel', '23rd & San Jacinto @ DKR Stadium',
       '26th & Nueces', '28th & Rio Grande', '2nd & Congress',
       '2nd & Lavaca @ City Hall', '3rd & Nueces',
       '3rd & Trinity @ The Convention Center', '3rd & West',
       '4th & Congress', '4th & Guadalupe @ Republic Square',
       '4th & Neches @ MetroRail Downtown', '4th & Sabine', '5th & Bowie',
       '5th & Campbell', '5th & Guadalupe @ Republic Square',
       '5th & San Marcos', '6th & Brazos', '6th & Chalmers',
   

In [ ]:
#Find pairs with a similarity greater than or equal to90%
matched_pairs = []

for loc1, loc2 in combinations(checkout_kiosk, 2):  # Compare all possible pairs
    similarity = fuzz.token_sort_ratio(loc1, loc2)
    if similarity >= 80:  # Only store exact matches
        matched_pairs.append((loc1, loc2))

#Display the results
for match in matched_pairs:
    print(match)

('10th & Red River', '8th & Red River')
('11th & Congress @ The Texas Capitol', 'Capitol Station & Congress & 11th')
('11th & San Jacinto', '8th & San Jacinto')
('12th & San Jacinto @ State Capitol Visitors Garage', 'State Capitol Visitors Garage @ San Jacinto & 12th')
('13th & San Antonio', '16th & San Antonio')
('13th & Trinity', '6th & Trinity')
('17th & Guadalupe', '21st & Guadalupe')
('17th & Guadalupe', 'Guadalupe & 21st')
('17th & Guadalupe', 'Guadalupe & 6th')
('21st & Guadalupe', 'Guadalupe & 21st')
('21st & Guadalupe', 'Guadalupe & 6th')
('22.5 & Rio Grande', '23rd & Rio Grande')
('22.5 & Rio Grande', '28th & Rio Grande')
('22.5 & Rio Grande', 'Rio Grande & 12th')
('22.5 & Rio Grande', 'Rio Grande & 28th')
('22nd & Pearl', '23rd & Pearl')
('23rd & Rio Grande', '28th & Rio Grande')
('23rd & Rio Grande', 'Rio Grande & 12th')
('23rd & Rio Grande', 'Rio Grande & 28th')
('26th & Nueces', 'Nueces & 26th')
('28th & Rio Grande', 'ACC - Rio Grande & 12th')
('28th & Rio Grande', 'Rio G

In [75]:
#Manually remove unwanted pairs
pairs_to_remove = [
        ('10th & Red River', '8th & Red River'),
    ('11th & San Jacinto', '8th & San Jacinto'),
    ('13th & San Antonio', '16th & San Antonio'),
    ('13th & Trinity', '6th & Trinity'),
    ('17th & Guadalupe', '21st & Guadalupe'),
    ('17th & Guadalupe', 'Guadalupe & 21st'),
    ('17th & Guadalupe', 'Guadalupe & 6th'),
    ('21st & Guadalupe', 'Guadalupe & 6th'),
    ('22.5 & Rio Grande', '23rd & Rio Grande'),
    ('22.5 & Rio Grande', '28th & Rio Grande'),
    ('22.5 & Rio Grande', 'Rio Grande & 12th'),
    ('22.5 & Rio Grande', 'Rio Grande & 28th'),
    ('22nd & Pearl', '23rd & Pearl'),
    ('23rd & Rio Grande', '28th & Rio Grande'),
    ('23rd & Rio Grande', 'Rio Grande & 12th'),
    ('23rd & Rio Grande', 'Rio Grande & 28th'),
    ('28th & Rio Grande', 'ACC - Rio Grande & 12th'),
    ('28th & Rio Grande', 'Rio Grande & 12th'),
    ('4th & Congress', '6th & Congress'),
    ('4th & Congress', '8th & Congress'),
    ('4th & Guadalupe @ Republic Square', '5th & Guadalupe @ Republic Square'),
    ('4th & Guadalupe @ Republic Square', 'Republic Square @ 5th & Guadalupe'),
    ('5th & Guadalupe @ Republic Square', 'Republic Square @ Guadalupe & 4th St.'),
    ('6th & Congress', '8th & Congress'),
    ('6th & Lavaca', '8th & Lavaca'),
    ('ACC - Rio Grande & 12th', 'Rio Grande & 12th'),
    ('ACC - Rio Grande & 12th', 'Rio Grande & 28th'),
    ('East 2nd & Pedernales', 'East 6th & Pedernales'),
    ('Guadalupe & 21st', 'Guadalupe & 6th'),
    ('Nash Hernandez & East @ RBJ South', 'Nash Hernandez @ RBJ South'),
    ('Republic Square @ 5th & Guadalupe', 'Republic Square @ Guadalupe & 4th St.'),
    ('Rio Grande & 12th', 'Rio Grande & 28th'),
    ('South Congress & James', 'South Congress & Mary'),
    ('Zilker Park', 'Zilker Park West')
]

#Filter matches
#Normalize function to ensure consistent formatting
def normalize(s):
    return s.lower().strip()

#Create a set of pairs to remove with normalization
pairs_to_remove_set = {(normalize(a), normalize(b)) for a, b in pairs_to_remove}

#Filter matches
filtered_matches = [
    pair for pair in matched_pairs 
    if (normalize(pair[0]), normalize(pair[1])) not in pairs_to_remove_set 
    and (normalize(pair[1]), normalize(pair[0])) not in pairs_to_remove_set
]

#Display the filtered results
for match in filtered_matches:
    print(match)

('11th & Congress @ The Texas Capitol', 'Capitol Station & Congress & 11th')
('12th & San Jacinto @ State Capitol Visitors Garage', 'State Capitol Visitors Garage @ San Jacinto & 12th')
('21st & Guadalupe', 'Guadalupe & 21st')
('26th & Nueces', 'Nueces & 26th')
('28th & Rio Grande', 'Rio Grande & 28th')
('2nd & Lavaca @ City Hall', 'City Hall & Lavaca & 2nd')
('3rd & Nueces', 'Nueces & 3rd')
('3rd & Nueces', 'Nueces @ 3rd')
('3rd & Trinity @ The Convention Center', 'Convention Center & 3rd & Trinity')
('4th & Guadalupe @ Republic Square', 'Republic Square @ Guadalupe & 4th St.')
('5th & Guadalupe @ Republic Square', 'Republic Square @ 5th & Guadalupe')
('6th & Brazos', 'Brazos & 6th')
('6th & Lavaca', 'Lavaca & 6th')
('6th & Trinity', 'Trinity & 6th St')
('6th & West', 'West & 6th St.')
('8th & Lavaca', 'Lavaca & 6th')
('8th & San Jacinto', 'San Jacinto & 8th Street')
('9th & Henderson', 'Henderson & 9th')
('Capital Metro HQ - East 5th at Broadway', 'East 5th & Broadway @ Capital Metro

In [76]:
#Define dictionary for changes
checkout_corrections = {
    'Capitol Station & Congress & 11th':'11th & Congress @ The Texas Capitol',
    'State Capitol Visitors Garage @ San Jacinto & 12th': '12th & San Jacinto @ State Capitol Visitors Garage',
    'Guadalupe & 21st': '21st & Guadalupe',
    'Nueces & 26th': '26th & Nueces',
    'Rio Grande & 28th': '28th & Rio Grande',
    'City Hall & Lavaca & 2nd': '2nd & Lavaca @ City Hall',
    'Nueces & 3rd':'Nueces @ 3rd',
    '3rd & Nueces': 'Nueces @ 3rd',
    'Convention Center & 3rd & Trinity': '3rd & Trinity @ The Convention Center',
    'Republic Square @ Guadalupe & 4th St.': '4th & Guadalupe @ Republic Square', 
    'Republic Square @ 5th & Guadalupe': '5th & Guadalupe @ Republic Square',
    'Brazos & 6th': '6th & Brazos',
    'Lavaca & 6th': '6th & Lavaca', 
    'Trinity & 6th St': '6th & Trinity',
    'West & 6th St.': '6th & West',
    'San Jacinto & 8th Street': '8th & San Jacinto',
    'Henderson & 9th': '9th & Henderson',
    'Capital Metro HQ - East 5th at Broadway':'East 5th & Broadway @ Capital Metro HQ',
    'East 11th St. & San Marcos': 'East 11th & San Marcos',
    'East 11th St. at Victory Grill':'East 11th & Victory Grill',
    'Medina & East 6th':'East 6th & Medina',
    'East 6th & Pedernales St.':'East 6th & Pedernales',
    'Lake Austin Blvd & Deep Eddy': 'Lake Austin Blvd @ Deep Eddy',
    'Lakeshore & Austin Hostel':'Lakeshore @ Austin Hostel',
    'South 1st & Riverside @ Long Center':'Long Center @ South 1st & Riverside',
    'Nueces & 3rd':'Nueces @ 3rd',
    'Rainey & Cummings':'Rainey St @ Cummings'
}


#Replace values
df_cleaned['checkout_kiosk'] = df_cleaned['checkout_kiosk'].replace(checkout_corrections)

#Check results
checkout_kiosk = df_cleaned['checkout_kiosk'].unique()
checkout_kiosk.sort()
checkout_kiosk

array(['10th & Red River', '11th & Congress @ The Texas Capitol',
       '11th & Salina', '11th & San Jacinto',
       '12th & San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th & Trinity',
       '13th & Trinity @ Waterloo Greenway', '16th & San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '21st & Guadalupe',
       '21st & Speedway @ PCL', '21st & University', '22.5 & Rio Grande',
       '22nd & Pearl', '23rd & Pearl', '23rd & Rio Grande',
       '23rd & San Gabriel', '23rd & San Jacinto @ DKR Stadium',
       '26th & Nueces', '28th & Rio Grande', '2nd & Congress',
       '2nd & Lavaca @ City Hall',
       '3rd & Trinity @ The Convention Center', '3rd & West',
       '4th & Congress', '4th & Guadalupe @ Republic Square',
       '4th & Neches @ MetroRail Downtown', '4th & Sabine', '5th & Bowie',
       '5th & Campbell', '5th & Guadalupe @ Republic Square',
       '5th & San Marcos', '6th & Brazos', '6th & Chalmers',
       '6th & Congr

In [77]:
#Return Kiosk
return_kiosk = df_cleaned['return_kiosk'].unique()
return_kiosk.sort()
return_kiosk

array(['10th & Red River', '11th & Congress @ The Texas Capitol',
       '11th & Salina', '11th & San Jacinto',
       '12th & San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th & Trinity',
       '13th & Trinity @ Waterloo Greenway', '16th & San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '21st & Guadalupe',
       '21st & Speedway @ PCL', '21st & Speedway @PCL',
       '21st & University', '22.5 & Rio Grande', '22nd & Pearl',
       '23rd & Pearl', '23rd & Rio Grande', '23rd & San Gabriel',
       '23rd & San Jacinto @ DKR Stadium', '26th & Nueces', '28th & Rio',
       '28th & Rio Grande', '2nd & Congress', '2nd & Lavaca @ City Hall',
       '3rd & Nueces', '3rd & Trinity @ The Convention Center',
       '3rd & West', '4th & Congress',
       '4th & Guadalupe @ Republic Square',
       '4th & Neches @ MetroRail Downtown', '4th & Sabine', '5th & Bowie',
       '5th & Campbell', '5th & Guadalupe @ Republic Square',
       '5th & San M

**Changes:**

In [78]:
#Replace `21st & Speedway @PCL` with `21st & Speedway @ PCL`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('21st & Speedway @PCL', '21st & Speedway @ PCL')

#Convert `Congress & Cesar Chavez` to `Cesar Chavez & Congress`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Congress & Cesar Chavez', 'Cesar Chavez & Congress')

#Convert `East 11th St. & San Marcos` to `East 11th & San Marcos`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('East 11th St. & San Marcos', 'East 11th & San Marcos')

#Convert `East 11th St. at Victory Grill` to `East 11th @ Victory Grill`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('East 11th St. at Victory Grill', 'East 11th @ Victory Grill')


#Convert `East 6th & Pedernales` to `East 6th & Pedernales St.`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('East 6th & Pedernales', 'East 6th & Pedernales St.')

#Convert `East 6th at Robert Martinez` to `East 6th & Robert T. Martinez`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('East 6th at Robert Martinez', 'East 6th & Robert T. Martinez')

#Convert `Lake Austin Blvd & Deep Eddy` to `Lake Austin Blvd @ Deep Eddy`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Lake Austin Blvd & Deep Eddy', 'Lake Austin Blvd @ Deep Eddy')

#Convert `Lakeshore & Austin Hostel` to `Lakeshore @ Austin Hostel`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Lakeshore & Austin Hostel', 'Lakeshore @ Austin Hostel')

#Convert `Nueces & 3rd` to `Nueces @ 3rd`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Nueces & 3rd', 'Nueces @ 3rd')

#Convert `Rainey St @ Cummings` to `Rainey @ Cummings` for consistency purposes
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Rainey St @ Cummings', 'Rainey @ Cummings')
df_cleaned['return_kiosk']  = df_cleaned['return_kiosk'].replace('Rainey & Cummings', 'Rainey @ Cummings')
#Convert `Riverside @ S. Lamar` to `Riverside & South Lamar`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Riverside @ S. Lamar', 'Riverside & South Lamar')

#Convert `South Congress & Barton Springs @ The Austin American-Statesman` to `South Congress & Barton Springs at the Austin American-Statesman`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('South Congress & Barton Springs at the Austin American-Statesman'
                                                                    ,'South Congress & Barton Springs @ The Austin American-Statesman')

#Convert `Sterzing & Barton Springs` to `Sterzing @ Barton Springs`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Sterzing & Barton Springs', 'Sterzing @ Barton Springs')
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Sterzing at Barton Springs', 'Sterzing @ Barton Springs')

#Convert `Barton Springs & Kinney` to `Barton Springs @ Kinney Ave`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Barton Springs & Kinney', 'Barton Springs @ Kinney Ave')

#Convert `East 11th & Victory Grill` to `East 11th @ Victory Grill`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('East 11th & Victory Grill', 'East 11th @ Victory Grill')

#Convert `Trinity & 6th Street` to `Trinity & 6th St`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Trinity & 6th Street', 'Trinity & 6th St')

#Convert `28th & Rio` to `28th & Rio Grande`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('28th & Rio', '28th & Rio Grande')

#Convert `Brazos & 6th` to `6th & Brazos`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Brazos & 6th', '6th & Brazos')

#Convert `Lavaca & 6th` to `6th & Lavaca`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Lavaca & 6th', '6th & Lavaca')

#Convert `Medina & East 6th` to `East 6th & Medina`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Medina & East 6th', 'East 6th & Medina')

#Convert `Nueces @ 3rd` to `3rd & Nueces`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Nueces @ 3rd','3rd & Nueces')

#Convert `Red River & 8th Street` to `8th & Red River`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Red River & 8th Street', '8th & Red River')

#Convert `Republic Square @ Guadalupe & 4th St.` to `4th & Guadalupe @ Republic Square`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Republic Square @ Guadalupe & 4th St.', '4th & Guadalupe @ Republic Square')

#Convert `San Jacinto & 8th Street` to `8th & San Jacinto`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('San Jacinto & 8th Street', '8th & San Jacinto')

#Convert `Henderson & 9th` to `9th & Henderson`
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace('Henderson & 9th', '9th & Henderson')

In [79]:
#Recheck Return Kiosk
return_kiosk = df_cleaned['return_kiosk'].unique()
return_kiosk.sort()
return_kiosk

array(['10th & Red River', '11th & Congress @ The Texas Capitol',
       '11th & Salina', '11th & San Jacinto',
       '12th & San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th & Trinity',
       '13th & Trinity @ Waterloo Greenway', '16th & San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '21st & Guadalupe',
       '21st & Speedway @ PCL', '21st & University', '22.5 & Rio Grande',
       '22nd & Pearl', '23rd & Pearl', '23rd & Rio Grande',
       '23rd & San Gabriel', '23rd & San Jacinto @ DKR Stadium',
       '26th & Nueces', '28th & Rio Grande', '2nd & Congress',
       '2nd & Lavaca @ City Hall', '3rd & Nueces',
       '3rd & Trinity @ The Convention Center', '3rd & West',
       '4th & Congress', '4th & Guadalupe @ Republic Square',
       '4th & Neches @ MetroRail Downtown', '4th & Sabine', '5th & Bowie',
       '5th & Campbell', '5th & Guadalupe @ Republic Square',
       '5th & San Marcos', '6th & Brazos', '6th & Chalmers',
   

In [ ]:
#Find pairs with a similarity greater than or equal to 90%
matched_pairs = []

for loc1, loc2 in combinations(return_kiosk, 2):  # Compare all possible pairs
    similarity = fuzz.token_sort_ratio(loc1, loc2)
    if similarity >= 80:  # Only store exact matches
        matched_pairs.append((loc1, loc2))

# Display the results
for match in matched_pairs:
    print(match)

('10th & Red River', '8th & Red River')
('11th & Congress @ The Texas Capitol', 'Capitol Station & Congress & 11th')
('11th & San Jacinto', '8th & San Jacinto')
('12th & San Jacinto @ State Capitol Visitors Garage', 'State Capitol Visitors Garage @ San Jacinto & 12th')
('13th & San Antonio', '16th & San Antonio')
('13th & Trinity', '6th & Trinity')
('17th & Guadalupe', '21st & Guadalupe')
('17th & Guadalupe', 'Guadalupe & 21st')
('17th & Guadalupe', 'Guadalupe & 6th')
('21st & Guadalupe', 'Guadalupe & 21st')
('21st & Guadalupe', 'Guadalupe & 6th')
('22.5 & Rio Grande', '23rd & Rio Grande')
('22.5 & Rio Grande', '28th & Rio Grande')
('22.5 & Rio Grande', 'Rio Grande & 12th')
('22.5 & Rio Grande', 'Rio Grande & 28th')
('22nd & Pearl', '23rd & Pearl')
('23rd & Rio Grande', '28th & Rio Grande')
('23rd & Rio Grande', 'Rio Grande & 12th')
('23rd & Rio Grande', 'Rio Grande & 28th')
('26th & Nueces', 'Nueces & 26th')
('28th & Rio Grande', 'ACC - Rio Grande & 12th')
('28th & Rio Grande', 'Rio G

In [89]:
#Manually remove unwanted pairs
pairs_to_remove = [
    ('10th & Red River', '8th & Red River'),
    ('11th & San Jacinto', '8th & San Jacinto'),
    ('13th & San Antonio', '16th & San Antonio'),
    ('13th & Trinity', '6th & Trinity'),
    ('17th & Guadalupe', '21st & Guadalupe'),
    ('17th & Guadalupe', 'Guadalupe & 21st'),
    ('17th & Guadalupe', 'Guadalupe & 6th'),
    ('21st & Guadalupe', 'Guadalupe & 6th'),
    ('22.5 & Rio Grande', '23rd & Rio Grande'),
    ('22.5 & Rio Grande', '28th & Rio Grande'),
    ('22.5 & Rio Grande', 'Rio Grande & 12th'),
    ('22.5 & Rio Grande', 'Rio Grande & 28th'),
    ('22nd & Pearl', '23rd & Pearl'),
    ('23rd & Rio Grande', '28th & Rio Grande'),
    ('23rd & Rio Grande', 'Rio Grande & 12th'),
    ('23rd & Rio Grande', 'Rio Grande & 28th'),
    ('28th & Rio Grande', 'ACC - Rio Grande & 12th'),
    ('28th & Rio Grande', 'Rio Grande & 12th'),
    ('28th & Rio Grande', 'Rio Grande & 28th'),
    ('4th & Congress', '6th & Congress'),
    ('4th & Congress', '8th & Congress'),
    ('4th & Guadalupe @ Republic Square', '5th & Guadalupe @ Republic Square'),
    ('6th & Congress', '8th & Congress'),
    ('6th & Lavaca', '8th & Lavaca'),
    ('ACC - Rio Grande & 12th', 'Rio Grande & 12th'),
    ('ACC - Rio Grande & 12th', 'Rio Grande & 28th'),
    ('Guadalupe & 21st', 'Guadalupe & 6th'),
    ('Nash Hernandez & East @ RBJ South', 'Nash Hernandez @ RBJ South'),
    ('Rio Grande & 12th', 'Rio Grande & 28th'),
    ('South Congress & James', 'South Congress & Mary'),
    ('Zilker Park', 'Zilker Park West')
]

#Filter matches
#Normalize function to ensure consistent formatting
def normalize(s):
    return s.lower().strip()

#Create a set of pairs to remove with normalization
pairs_to_remove_set = {(normalize(a), normalize(b)) for a, b in pairs_to_remove}

#Filter matches
filtered_matches = [
    pair for pair in matched_pairs 
    if (normalize(pair[0]), normalize(pair[1])) not in pairs_to_remove_set 
    and (normalize(pair[1]), normalize(pair[0])) not in pairs_to_remove_set
]

#Display the filtered results
for match in filtered_matches:
    print(match)

('11th & Congress @ The Texas Capitol', 'Capitol Station & Congress & 11th')
('12th & San Jacinto @ State Capitol Visitors Garage', 'State Capitol Visitors Garage @ San Jacinto & 12th')
('21st & Guadalupe', 'Guadalupe & 21st')
('26th & Nueces', 'Nueces & 26th')
('2nd & Lavaca @ City Hall', 'City Hall & Lavaca & 2nd')
('3rd & Trinity @ The Convention Center', 'Convention Center & 3rd & Trinity')
('4th & Guadalupe @ Republic Square', 'Republic Square @ 5th & Guadalupe')
('5th & Guadalupe @ Republic Square', 'Republic Square @ 5th & Guadalupe')
('6th & Trinity', 'Trinity & 6th St')
('6th & West', 'West & 6th St.')
('Capital Metro HQ - East 5th at Broadway', 'East 5th & Broadway @ Capital Metro HQ')
('Long Center @ South 1st & Riverside', 'South 1st & Riverside @ Long Center')


In [90]:
#Define dictionary for changes
return_corrections = {
    'Capitol Station & Congress & 11th': '11th & Congress @ The Texas Capitol',
    'State Capitol Visitors Garage @ San Jacinto & 12th': '12th & San Jacinto @ State Capitol Visitors Garage',
    'Guadalupe & 21st': '21st & Guadalupe',
    'Nueces & 26th': '26th & Nueces',
    'City Hall & Lavaca & 2nd': '2nd & Lavaca @ City Hall',
    'Convention Center & 3rd & Trinity': '3rd & Trinity @ The Convention Center',
    'Republic Square @ 5th & Guadalup': '5th & Guadalupe @ Republic Square',
    'Trinity & 6th St': '6th & Trinity',
    'West & 6th St.': '6th & West',
    'Capital Metro HQ - East 5th at Broadway': 'East 5th & Broadway @ Capital Metro HQ',
    'South 1st & Riverside @ Long Center': 'Long Center @ South 1st & Riverside',
    'Rio Grande & 28th': '28th & Rio Grande'
}

#Replace values
df_cleaned['return_kiosk'] = df_cleaned['return_kiosk'].replace(return_corrections)

#Check results
return_kiosk = df_cleaned['return_kiosk'].unique()
return_kiosk.sort()
return_kiosk

array(['10th & Red River', '11th & Congress @ The Texas Capitol',
       '11th & Salina', '11th & San Jacinto',
       '12th & San Jacinto @ State Capitol Visitors Garage',
       '13th & San Antonio', '13th & Trinity',
       '13th & Trinity @ Waterloo Greenway', '16th & San Antonio',
       '1701 Barton Springs Road', '17th & Guadalupe', '21st & Guadalupe',
       '21st & Speedway @ PCL', '21st & University', '22.5 & Rio Grande',
       '22nd & Pearl', '23rd & Pearl', '23rd & Rio Grande',
       '23rd & San Gabriel', '23rd & San Jacinto @ DKR Stadium',
       '26th & Nueces', '28th & Rio Grande', '2nd & Congress',
       '2nd & Lavaca @ City Hall', '3rd & Nueces',
       '3rd & Trinity @ The Convention Center', '3rd & West',
       '4th & Congress', '4th & Guadalupe @ Republic Square',
       '4th & Neches @ MetroRail Downtown', '4th & Sabine', '5th & Bowie',
       '5th & Campbell', '5th & Guadalupe @ Republic Square',
       '5th & San Marcos', '6th & Brazos', '6th & Chalmers',
   

## 4. Check for duplicates

In [91]:
#Check for duplicates
duplicates = df_cleaned.duplicated()
duplicates.value_counts()

False    2270239
Name: count, dtype: int64

**No duplicate values.**

## 5. Check Trip Duration column for outliers

In [83]:
#Set display option to avoid scientific notation
pd.set_option('display.float_format', '{:.6f}'.format)

In [84]:
#Check out statistics
df_cleaned[['trip_duration_minutes']].describe()

,trip_duration_minutes
count,2270239.000000
mean,28.668326
std,124.540938
min,2.000000
25%,6.000000
50%,11.000000
75%,27.000000
max,34238.000000


In [85]:
#Find the row with the longest trip duration 
df_cleaned[df_cleaned['trip_duration_minutes'] == df_cleaned['trip_duration_minutes'].max()]

,trip_id,membership_type,bicycle_id,bike_type,checkout_datetime,checkout_date,checkout_time,checkout_kiosk_id,checkout_kiosk,return_kiosk_id,return_kiosk,trip_duration_minutes,month,year,time_of_day,day_of_week,checkout_day,hour,time_period,checkout_kisok
1561656,20465659,Single Trip Ride,893,classic,08/07/2019 10:25:19 PM,2019-08-07,22:25:19,4062,Lakeshore & Pleasant Valley,2566,Electric Drive & Sandra Muraida Way @ Pfluger ...,34238,8,2019,PM,Wednesday,7,22,Late Night,Lakeshore & Pleasant Valley


In [92]:
#Filter out trips longer than 1 day
df_trips = df_cleaned[df_cleaned['trip_duration_minutes'] <= 1440]
df_trips

,trip_id,membership_type,bicycle_id,bike_type,checkout_datetime,checkout_date,checkout_time,checkout_kiosk_id,checkout_kiosk,return_kiosk_id,return_kiosk,trip_duration_minutes,month,year,time_of_day,day_of_week,checkout_day,hour,time_period,checkout_kisok
0,16459203,Local365,37,classic,12/01/2017 09:51:41 AM,2017-12-01,09:51:41,2496,8th & Congress,2495,4th & Congress,3,12,2017,AM,Friday,1,9,Off-Peak,8th & Congress
1,4106422,Local365,971,classic,03/13/2015 04:20:24 PM,2015-03-13,16:20:24,2495,4th & Congress,2495,4th & Congress,14,3,2015,PM,Friday,13,16,Evening Rush,4th & Congress
2,3454605,Local365,29,classic,10/14/2014 08:33:54 AM,2014-10-14,08:33:54,2712,Toomey Rd @ South Lamar,2575,Riverside & South Lamar,22,10,2014,AM,Tuesday,14,8,Morning Commute,Toomey Rd @ South Lamar
3,3454592,Local365,646,classic,10/14/2014 08:32:03 AM,2014-10-14,08:32:03,2552,3rd & West,2496,8th & Congress,29,10,2014,AM,Tuesday,14,8,Morning Commute,3rd & West
4,4106421,24 Hour Walk Up Pass,124,classic,03/13/2015 04:20:15 PM,2015-03-13,16:20:15,2498,Dean Keeton & Speedway,2549,Long Center @ South 1st & Riverside,10,3,2015,PM,Friday,13,16,Evening Rush,Dean Keeton & Speedway
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271148,32559582,Local31,17370,electric,02/24/2024 05:52:12 PM,2024-02-24,17:52:12,2501,5th & Bowie,2707,Rainey @ Cummings,15,2,2024,PM,Saturday,24,17,Evening Rush,5th & Bowie
2271149,32490697,Student Membership,18331,electric,02/15/2024 03:00:58 PM,2024-02-15,15:00:58,7125,23rd & Pearl,2498,Dean Keeton & Speedway,5,2,2024,PM,Thursday,15,15,Afternoon Surge,23rd & Pearl
2271150,32531779,Local31,21457,electric,02/21/2024 03:41:48 PM,2024-02-21,15:41:48,2539,3rd & Trinity @ The Convention Center,2574,Zilker Park,19,2,2024,PM,Wednesday,21,15,Afternoon Surge,3rd & Trinity @ The Convention Center
2271151,32601620,Student Membership,19608,electric,02/29/2024 08:01:55 PM,2024-02-29,20:01:55,3795,Dean Keeton & Whitis,3798,21st & Speedway @ PCL,4,2,2024,PM,Thursday,29,20,Evening Leisure,Dean Keeton & Whitis


## 6. Save cleaned data

In [95]:
#Save cleaned version of csv
df_trips.to_csv('Cleaned_Austin_CapMetroBike_Trips.csv',index=False)